In [1]:
!pip install PyPDF2

In [2]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz --no-deps

     |████████████████████████████████| 12.0 MB 5.0 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047106 sha256=d76d92a5ff186a95328ce49a9229173d66bc5a2a08c55ce7eb1887d91cfb81bd
  Stored in directory: /Users/shounakmondal/Library/Caches/pip/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm


In [11]:
!pip install pyenchant

     |████████████████████████████████| 55 kB 828 kB/s eta 0:00:01


In [12]:
import spacy
import PyPDF2
import os

In [13]:
# spacy english model (large)
nlp = spacy.load('en_core_web_lg')

In [71]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io

# method for reading a pdf file
def readPdfFile(filename, folder_name):
    
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open('./docs/doc.pdf', 'rb') as fh:

        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()


    return text

In [15]:
# customer sentence segmenter for creating spacy document object
def setCustomBoundaries(doc):
    # traversing through tokens in document object
    for token in doc[:-1]:
        if token.text == ';':
            doc[token.i + 1].is_sent_start = True
        if token.text == ".":
            doc[token.i + 1].is_sent_start = False
    return doc


In [16]:
# create spacy document object from pdf text
def getSpacyDocument(pdf_text, nlp):
    main_doc = nlp(pdf_text)  # create spacy document object

    return main_doc

In [17]:
# adding setCusotmeBoundaries to the pipeline
nlp.add_pipe(setCustomBoundaries, before='parser')

In [312]:
from scipy import spatial

import enchant
d = enchant.Dict("en_US")

from spacy.tokens import Doc

# convert keywords to vector
def createKeywordsVectors(keyword, nlp):
    print("Searching document for word: ",keyword)
    doc = nlp(keyword)  # convert to document object
    #doc = Doc(nlp.vocab, words = keyword)  # convert to document object
    #print("doc",doc.vector)
    return doc.vector


# method to find cosine similarity
def cosineSimilarity(vect1, vect2):
    # return cosine distance
    return 1 - spatial.distance.cosine(vect1, vect2)


# method to find similar words
def getSimilarWords(keyword, nlp):
    similarity_list = []

    keyword_vector = createKeywordsVectors(keyword, nlp)

    for tokens in nlp.vocab:
        if (tokens.has_vector):
            if (tokens.is_lower):
                if (tokens.is_alpha):
                    similarity_list.append((tokens, cosineSimilarity(keyword_vector, tokens.vector)))

    similarity_list = sorted(similarity_list, key=lambda item: -item[1])
    similarity_list = similarity_list[:30]
    top_similar_words = [item[0].text for item in similarity_list]
    top_similar_words = top_similar_words[:3]
    top_similar_words.append(keyword)
    #print("top_similar_words:1",top_similar_words)
#     for token in Doc(nlp.vocab, words = keyword):
#         top_similar_words.insert(0, token.lemma_)

#     for words in top_similar_words:
#         if words.endswith("s"):
#             top_similar_words.append(words[0:len(words)-1])

#     top_similar_words = list(set(top_similar_words))
    #print("top_similar_words:",top_similar_words)
    #top_similar_words = [words for words in top_similar_words if enchant_dict.check(words) == True]
    top_similar_words = [words for words in top_similar_words if d.check(words) == True]
    
    top_similar_words = [words for words in top_similar_words]
    similarity_list.clear()
    return ", ".join(top_similar_words)


In [313]:
#keywords = ["safety","guidelines","manufacturers"]
keywords = "total cost"
similar_keywords = getSimilarWords(keywords, nlp)
print("Similar words are:",similar_keywords)

Searching document for word:  total cost
Similar words are: total, cost, costs


In [314]:
from spacy.matcher import PhraseMatcher
from scipy import spatial

# method for searching keyword from the text
def search_for_keyword(keyword, doc_obj, nlp):
    phrase_matcher = PhraseMatcher(nlp.vocab)
    li = list(keyword.split(",")) 
    patterns = [nlp.make_doc(text) for text in li]
    phrase_matcher.add("TerminologyList", patterns)

    matched_items = phrase_matcher(doc_obj)
    matched_text = []
    for match_id, start, end in matched_items:
        text = nlp.vocab.strings[match_id]
        span = doc_obj[start: end]
        matched_text.append(span.sent.text)
    
    for txt in matched_text:
        print("Found Match ####")
        print(txt)
    

In [315]:
pdf = readPdfFile("testdoc.pdf","docs")

In [316]:
doc_obj = getSpacyDocument(pdf,nlp)

In [318]:
print("Searching for...",similar_keywords,"\n")
search_for_keyword(similar_keywords,doc_obj, nlp)

Searching for... total, cost, costs 

Found Match ####
Comprised of leading researchers investigating 
various  dimensions  of  access  to  justice  and  cost  across  the  country,  
Found Match ####
the  costs  of  justice,  
Found Match ####
the  monetary  costs  associated  with 
experiencing  everyday  legal  problems.  
Found Match ####
the  monetary  costs  to 

Found Match ####
individuals  of  attempting  to  resolve  problems,  intangible  costs  associated  with  experiencing 

Found Match ####
In sum, 
respondents  were  asked  about  costs  
Found Match ####
212  adults  using  cell  phones  (for  a  total  of  3,263  adult  Canadians).    
Found Match ####
the  individual  experiences  and  costs  of  everyday  
Found Match ####
addressing  the  costs  of  everyday  
Found Match ####
But  what  costs,  and  how  much?    
Found Match ####
These 
intangible  costs  can  include,  for  example,  decreasing  physical  health,  high 
levels of stress and emotional problems, a